# _CommonCrawl日本語データの分割抽出

 このNotebookは, CommonCrawl日本語データ分割抽出を行うものです.

GENIACプロジェクトのTeam Hatakeyama(仮)ではCommonCrawlのデータを

質良く加工して日本語LLMの学習に役立つコーパス作成を進めています.

そのコーパス作成で問題となるポイントとして,

CommonCrawlのデータが多すぎることがあります(なんと100TB程度!).


 多くの言語が集まるデータから日本語のデータのみを抜く工程

でもデータ数の多さからチームメンバーだけで行うことは困難です.

そのため, チーム内外でCommonCrawlからの日本語データを分割で抽出し,

最終的に統合するということを目指しています.


## 手順 (Google Colab)..

今回CommonCrawlにおける90000個のアーカイブデータ(warc)を分割で処理します.

90000個を10個ずつのバッチに分けて処理, その結果をGoogleDriveに格納することで,

90000個すべての処理を目指します.

そして, その上で行っていただきたいのは,

1)このセル以降のセルを上から順番に実行していく.

2)最後のセルにあるbatch_numberを変更して, セルを実行
https://colab.research.google.com/drive/1Gq8HQ0iyASH5iOAkosclJEQTwYJvYRmy#scrollTo=UawI0uZgAjz6&line=3&uniqifier=1

3)ファイルメニューに現れる/submit/に保存されている{batch_number}.gzファイルを
ダウンロード

4)バッチの処理が終わったことをgenaic slackなどで共有いただき, アップロードするDriveの場所の指示を受けてください.

60分程度実行すると, 1バッチ終了します.


### 手順 (個別環境 linux)

1)このGoogle Colab Notebookをダウンロードしてください.

2)jupyterが使用できるpython環境で実行してください.

## Environment

pythonのライブラリをインストールします.

※初回だけ実行してください

In [12]:
!pip install warcio
!pip install beautifulsoup4
!pip install Jupyter-Beeper
!pip install lxml
!pip install boto3

### Function - File Utils

今回の処理に必要な関数(ファイル処理関係)をインストールします.

※初回だけ実行してください

In [15]:

from botocore.exceptions import NoCredentialsError
import boto3
import os
import glob

import gzip
import shutil
import requests

base_url = "https://data.commoncrawl.org/"
os.makedirs("data/gz", exist_ok=True)
os.makedirs("data/warc", exist_ok=True)

def download_file(url, save_path):

    response = requests.get(url, stream=True)

    if response.status_code == 200:
        with open(save_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)
        print(f"ファイルが正常にダウンロードされました: {save_path}")
    else:
        print(f"ファイルのダウンロードに失敗しました。ステータスコード: {response.status_code}")


def decompress_gz(gz_path, output_path, remove_gz=True, fill_blank_gz=False):
    with gzip.open(gz_path, 'rb') as f_in:
        with open(output_path, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
    print(f"{gz_path}が解凍され、{output_path}に保存されました。")
    if remove_gz:
        os.remove(gz_path)

    if fill_blank_gz:
        with open(gz_path, 'w') as f:
            f.write("")

def get_cc_path_list(path_dir="data/path_list/*"):
    path_list = []
    for file_path in glob.glob(path_dir):
        print(file_path)
        with open(file_path, "r") as f:
            temp_path_list = f.readlines()

        temp_path_list = [path.strip() for path in temp_path_list]

        path_list += temp_path_list

    return path_list


def cc_path_to_urls(cc_path):
    url = base_url+cc_path
    filename = cc_path.replace("/", "_")
    gz_path = f"data/gz/{filename}"
    warc_path = f"data/warc/{filename}".replace(".gz", "")

    return url, gz_path, warc_path


def download_warc_file(path):
    url, gz_path, warc_path = cc_path_to_urls(path)

    if os.path.exists(warc_path):
        print(f"warc_pathにはファイルが存在しています")
        return warc_path
    try:
        if os.path.exists(gz_path):
            print(f"gz_pathがすでに存在します: {gz_path}")
        else:
            print("downloading "+url)
            download_file(url, gz_path)
        print("decompressing "+gz_path)
        decompress_gz(gz_path, warc_path,
                      remove_gz=False, fill_blank_gz=True)
        return warc_path
    except Exception as e:
        print(e)
        print("fail loading "+url)
        return warc_path
def download_file_with_progress(bucket_name, path, destination_filename):
    '''
    S3からファイルをダウンロードする関数
    '''
    from google.colab import userdata

    s3 = boto3.client('s3',
                      aws_access_key_id=userdata.get("AWS_ACCOUNT_ID"),
                      aws_secret_access_key=userdata.get("AWS_SECRET_KEY"),
                      region_name="us-east-1"
                      )
    # S3からファイルのサイズを取得
    response = s3.head_object(Bucket=bucket_name, Key=path)
    total_size = response['ContentLength']

    # tqdmを使ってプログレスバーを表示しながらファイルをダウンロード
    with tqdm(total=total_size, unit='B', unit_scale=True) as t:
        s3.download_file(bucket_name, path, destination_filename)

def download_warc_file_with_s3(path:str):
    '''前提条件：AWSの認証情報が設定されている必要がある'''
    '''s3からboto3を使用してダウンロードする.
    ,httpバージョンはdownload_warc_fileを使用する.S3のほうがHTTP
    よりも高速にダウンロードできる'''

    url, gz_path, warc_path = cc_path_to_urls(path)

    if os.path.exists(warc_path):
        print(f"warc_pathにはファイルが存在しています")
        return warc_path
    try:
        CC_BUCKET_NAME = "commoncrawl"
        print(f'download from s3://{CC_BUCKET_NAME}/{path} to {warc_path}')
        # s3からダウンロードする設定
        download_file_with_progress(CC_BUCKET_NAME, path, warc_path)

        decompress_gz(gz_path, warc_path,
                      remove_gz=False, fill_blank_gz=True)
        return warc_path
    except NoCredentialsError:
        print("Credentials not available")
    except Exception as e:
        print(e)
        print("fail loading " + url)
        return warc_path



### Function - Process Utils

今回の処理に必要な関数(warcファイルの処理)をインストールします.

※初回だけ実行してください

In [16]:
import os
import glob
import json
from tqdm import tqdm

import jupyter_beeper
from bs4 import BeautifulSoup
from warcio.archiveiterator import ArchiveIterator

def halfwidth_ratio(s):
    if len(s) == 0:  # 空の文字列の場合は0を返す
        return 0
    halfwidth_count = sum(
        1 for char in s
        if '\u0020' <= char <= '\u007E' or  # 基本的なASCII範囲
           '\uFF61' <= char <= '\uFF9F' or  # 半角カタカナ
           char in ('\u0009', '\u000A', '\u000D')  # タブ、改行、復帰
    )
    return halfwidth_count / len(s)


def pre_clean(soup):
    texts_with_tags = []
    for tag in soup.find_all(True):
        # 特定のタグを除外する場合
        # if tag.name not in ['html', 'body', 'ul']:
        text = tag.get_text(separator="\n", strip=True)
        spl_text = text.split("\n")
        spl_text = [i.strip() for i in spl_text if i.strip()]  # 空の文字列を除外
        for item in spl_text:
            if tag.name == "script" or tag.name == "style":
                continue
            texts_with_tags.append((item, tag.name))  # テキストとタグの名前をタプルとして追加
    return texts_with_tags


def extract_japanese_from_warc(path,
                               save_dir="json",
                               max_num=10**10,
                               ):
    ja_soup_list = []
    path = path.replace("\\", "/")  # for windows env
    filename = path.split("/")[-1].replace(".warc", ".json")
    if os.path.exists(f"{save_dir}/{filename}"):
        print("already done")
        return
    # 途中から再開する用の位置情報の取得
    if len(ja_soup_list) > 0:
        fin_record_id = ja_soup_list[-1]["record_id"]
    else:
        fin_record_id = 0
    # WARCファイルを開く
    record_id = 0
    with open(path, 'rb') as stream:
        for record in tqdm(ArchiveIterator(stream)):
            record_id += 1
            if record_id <= fin_record_id:
                continue
            if record.rec_type == 'response':
                if record.http_headers.get_header('Content-Type') == 'text/html':
                    content = record.content_stream().read()
                    soup = BeautifulSoup(content, 'lxml')
                    # <html>タグからlang属性を取得
                    html_tag = soup.find('html')
                    if html_tag and html_tag.has_attr('lang'):
                        lang = html_tag['lang']
                        texts = pre_clean(soup)
                        if len(texts) == 0:
                            continue
                        if lang == "ja":
                            if soup.title is not None:
                                title = soup.title.string
                            else:
                                title = ""
                            d = {
                                "record_id": record_id,
                                "url": record.rec_headers.get_header('WARC-Target-URI'),
                                "title": title,
                                "timestamp": record.rec_headers.get_header('WARC-Date'),
                                "text": texts,
                            }
                            ja_soup_list.append(d)
                        if len(ja_soup_list) > max_num:
                            break
    return ja_soup_list

def download_and_parse(cc_path, base_dir=None):
    # warcファイルのダウンロード:S3からダウンロードする場合はdownload_warc_file_with_s3を使用
    warc_path=download_warc_file_with_s3(cc_path)
    # ファイル関連の処理
    os.makedirs(base_dir, exist_ok=True)
    # パス関連の処理
    file_name = os.path.basename(warc_path)
    base_name = os.path.splitext(file_name)[0]
    file_base_name = "_".join(base_name.split("_")[2:])
    if base_dir is None:
        base_dir = "/tmp/"
    save_gz_path = f"{base_dir}/{file_base_name}_japanese.json.gz"
    try:
        tag_records = extract_japanese_from_warc(warc_path)
        is_error = False
        error_text = ""
    except Exception as e:
        tag_records = []
        is_error = True
        print(e)
        error_text = str(e)
    # 保存用のdictを作製
    save_dict = {
      "tag_records" : tag_records,
      "is_error" : is_error,
      "cc_path" : cc_path,
      "warc_path" : warc_path,
      "error_text" : error_text
    }
    with gzip.open(save_gz_path, 'wt', encoding="utf-8") as zipfile:
       json.dump(save_dict, zipfile, indent=2, ensure_ascii=False)
    return

def curation(batch_number, submit_dir="/content/submit", is_debug=False):
    cc_path_list = get_cc_path_list()
    if is_debug:
        n_batch = 3
    else:
        n_batch = 10
    start_idx, end_idx = batch_number * n_batch, (batch_number+1) * n_batch
    target_path_list  = cc_path_list[start_idx:end_idx]
    for cc_path in tqdm(target_path_list):
        download_and_parse(cc_path, f"process/batch{batch_number}")
    shutil.make_archive(f'{submit_dir}/{batch_number}',
                        format='zip', root_dir=f"process/batch{batch_number}")

    shutil.rmtree("process/")

# Beep when finished
def beep(frequency=2500):
    b = jupyter_beeper.Beeper()
    b.beep(frequency, secs=1, blocking=True)


## Data Preprocess

### Step1 Download CommonCrawl Paths

CommonCrawlにおけるアーカイブ(warc)が保存されているパスの文字列が

圧縮されたファイル(gz)で保存されている.

このファイルをダウンロードして, 解凍, data/data_list配下に保存する


In [17]:
"""
download path list from commoncrawl
"""
# Parameter
# 今回処理するwarcのパスリストが圧縮されているURL
# CC-MAIN-2023-50以外にも存在するが, 一旦このURLのみで行う
path_urls = [
    "https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-50/warc.paths.gz",
]
# パスリストをダウンロードするフォルダの作成
os.makedirs("data", exist_ok=True)
os.makedirs("data/path_list", exist_ok=True)

# Process
# Parameterで指定したURLからパス(gz)をダウンロードし,解凍する
for url in path_urls:
    file_name = url.split("/")[-2]+".gz"
    try:
        # パスリストが格納されているgzファイルをdata_list配下に保存
        download_file(url, f"data/path_list/{file_name}")
        # 保存されたgzファイルを解凍する
        decompress_gz(f"data/path_list/{file_name}",
                      f"data/path_list/{os.path.splitext(file_name)[0]}")
    except:
        pass


ファイルが正常にダウンロードされました: data/path_list/CC-MAIN-2023-50.gz
data/path_list/CC-MAIN-2023-50.gzが解凍され、data/path_list/CC-MAIN-2023-50に保存されました。


### step2 Check Step1 Result

warcのパスの文字列が保存されているかを確認する

※step1で異常があった際に実行してください

In [18]:
# Process
# 保存されているwarcファイルのパスのリストを取得
cc_paths = get_cc_path_list(path_dir="data/path_list/*")
# 表示
cc_paths

data/path_list/CC-MAIN-2023-50


['crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00000.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00001.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00002.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00003.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00004.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00005.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00006.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231128113443-00007.warc.gz',
 'crawl-data/CC-MAIN-2023-50/segments/1700679099281.67/warc/CC-MAIN-20231128083443-20231

### Step3 : Download Warc and Extract Japanese Site Data

warc.gzファイルのURI(その一部)を元にwarc.gzファイルをダウンロード/解凍(->warc)する.

warcファイルを読み込み, 内部に保存されている日本語サイトのデータのみを抽出する.

抽出した結果をjson.gz形式で圧縮して保存.

warcファイルは90000個ほど存在するため, この一部のみを処理していく必要あり,

そのため, 処理するファイルのパスをいくつかのbatchに分けている.

このbatchの番号(batch_number)を指定し, そのbatchにおける日本語ページを取得

取得した結果をまとめたzipファイルがsubmit/{batch_number}.zipに保存される.

保存されている内容を指定のGoogleDriveに配置ください

---

#### 行っていただく内容

- is_debugをTrueにして動くかを確認 (初回のみ)

- batch_numberを変更 (取りくむbatchをご指定ください)

- is_debugをFalseにしてデータ抽出/加工スタート

#### 2024/03/04 更新

結果のzipファイルを自動的にダウンロードするように変更いたしました

In [ ]:
import os
import shutil
import time

import jupyter_beeper
from google.colab import files

# 処理結果を自動的にダウンロードするように変更

#@markdown - ここの番号を指定を受けた番号に変更をしてください
#@markdown - batch_numberはコンマ区切りで入力してください
batch_number = 503,504, #@param
is_debug = False

# 保存用ディレクトリの指定
submit_dir = "submit"
# もしdriveがマウントできれば,上のsubmit_dirをコメントアウト, 以下をコードにしてください.
# submit_dir = "/content/drive/MyDrive/CommonCrawl/"


for num in batch_number:
  # batchの番号に従って,データの処理
  curation(num, submit_dir=submit_dir, is_debug=is_debug)

  # ファイルのダウンロード
  try:
    files.download(f"./submit/{num}.zip")
  except:
    print("ERROR: File Download Unsuccessful.")
  time.sleep(10)
  beep()
  shutil.rmtree("/content/data/gz")
  shutil.rmtree("/content/data/warc")
  os.mkdir("/content/data/gz")
  os.mkdir("/content/data/warc")

beep(5000)


data/path_list/CC-MAIN-2023-50


  0%|          | 0/10 [00:00<?, ?it/s]

warc_pathにはファイルが存在しています



0it [00:00, ?it/s]
33it [00:00, 251.20it/s]
75it [00:00, 325.50it/s]
109it [00:00, 329.39it/s]
243it [00:00, 264.53it/s]
330it [00:00, 362.76it/s]
420it [00:01, 421.89it/s]
473it [00:01, 375.53it/s]<ipython-input-16-8b963feab59b>:62: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  soup = BeautifulSoup(content, 'lxml')

582it [00:01, 513.50it/s]
705it [00:01, 485.99it/s]
853it [00:01, 668.33it/s]
940it [00:01, 697.49it/s]
1101it [00:02, 868.63it/s]
1238it [00:02, 987.15it/s]
1449it [00:02, 1260.40it/s]
1589it [00:02, 1014.05it/s]
1764it [00:02, 1169.29it/s]
1897it [00:02, 1104.17it/s]
2034it [00:02, 1139.24it/s]
21

download from s3://commoncrawl/crawl-data/CC-MAIN-2023-50/segments/1700679100047.66/warc/CC-MAIN-20231129010302-20231129040302-00531.warc.gz to data/warc/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00531.warc


...


0it [00:00, ?it/s]
66it [00:00, 244.74it/s]
91it [00:00, 244.43it/s]
116it [00:00, 185.45it/s]
135it [00:00, 165.80it/s]
189it [00:00, 234.24it/s]
214it [00:01, 201.09it/s]
249it [00:01, 197.37it/s]
363it [00:01, 397.77it/s]
413it [00:01, 354.49it/s]
456it [00:01, 262.15it/s]
491it [00:02, 175.33it/s]
524it [00:02, 196.86it/s]
553it [00:03, 102.39it/s]
582it [00:03, 114.46it/s]
602it [00:03, 122.53it/s]
679it [00:03, 216.75it/s]
716it [00:03, 139.93it/s]
762it [00:05, 58.78it/s] 
855it [00:05, 105.57it/s]
894it [00:11, 25.27it/s] 
963it [00:11, 38.56it/s]
1101it [00:11, 75.95it/s]
1206it [00:11, 106.45it/s]
1329it [00:12, 159.99it/s]
1440it [00:12, 219.34it/s]
1519it [00:12, 261.62it/s]
1692it [00:12, 401.70it/s]
1783it [00:12, 459.39it/s]
1947it [00:12, 634.93it/s]
2072it [00:12, 743.74it/s]
2186it [00:13, 415.69it/s]
2280it [00:13, 476.49it/s]
2378it [00:13, 552.72it/s]
2507it [00:13, 684.10it/s]
2609it [00:13, 554.15it/s]
2691it [00:14, 576.85it/s]
2769it [00:14, 584.69it/s]
2853it

download from s3://commoncrawl/crawl-data/CC-MAIN-2023-50/segments/1700679100047.66/warc/CC-MAIN-20231129010302-20231129040302-00535.warc.gz to data/warc/crawl-data_CC-MAIN-2023-50_segments_1700679100047.66_warc_CC-MAIN-20231129010302-20231129040302-00535.warc



  0%|          | 0.00/1.25G [00:16<?, ?B/s]


name 'progress' is not defined
fail loading https://data.commoncrawl.org/crawl-data/CC-MAIN-2023-50/segments/1700679100047.66/warc/CC-MAIN-20231129010302-20231129040302-00535.warc.gz



0it [00:00, ?it/s]
52it [00:00, 518.79it/s]
104it [00:00, 466.63it/s]
159it [00:00, 475.37it/s]
309it [00:00, 827.30it/s]
420it [00:00, 882.13it/s]
509it [00:01, 321.26it/s]
573it [00:02, 134.59it/s]
651it [00:02, 178.94it/s]
779it [00:02, 277.41it/s]
867it [00:02, 336.88it/s]
943it [00:03, 361.21it/s]
1053it [00:03, 336.06it/s]
1109it [00:03, 355.46it/s]
1194it [00:03, 377.87it/s]
1296it [00:03, 452.98it/s]
1353it [00:03, 454.99it/s]
1419it [00:04, 473.20it/s]
1526it [00:04, 599.68it/s]
1668it [00:04, 783.03it/s]
1833it [00:04, 997.16it/s]
1946it [00:04, 472.59it/s]
2031it [00:05, 409.11it/s]
2103it [00:05, 440.37it/s]
2184it [00:05, 500.19it/s]
2255it [00:05, 497.46it/s]
2370it [00:05, 621.69it/s]
2502it [00:05, 767.87it/s]
2595it [00:05, 803.18it/s]
2688it [00:06, 727.98it/s]
2790it [00:06, 782.55it/s]
2876it [00:06, 508.14it/s]
2997it [00:06, 634.89it/s]
3080it [00:06, 640.95it/s]
3158it [00:06, 668.48it/s]
3236it [00:07, 333.85it/s]
3295it [00:07, 221.49it/s]
3399it [00:08, 303.9